# Setup Env

In [5]:
# local install
# !pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu124 --user

# python env install
# !pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu124

In [6]:
# if versions error between torchvision and torch, uninstall and reinstall together
# !pip uninstall torch torchvision

In [12]:
# !pip install ultralytics

## Check GPU

In [1]:
!nvidia-smi

Wed Mar 26 22:05:35 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 566.14                 Driver Version: 566.14         CUDA Version: 12.7     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                  Driver-Model | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 4060 ...  WDDM  |   00000000:01:00.0 Off |                  N/A |
| N/A   38C    P4             13W /   30W |       0MiB /   8188MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

### Check if torch CUDA detects GPU

In [1]:
import torch
print("CUDA Available:", torch.cuda.is_available())
print("Device Count:", torch.cuda.device_count())
print("Device Name:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU")

CUDA Available: True
Device Count: 1
Device Name: NVIDIA GeForce RTX 4060 Laptop GPU


### Check Torch and Torchvision compability (nms)

In [2]:
from torchvision.ops import nms
boxes = torch.tensor([[0, 0, 1, 1], [0, 0, 2, 2]], dtype=torch.float32)
scores = torch.tensor([0.5, 0.9])
iou_threshold = 0.5
result = nms(boxes, scores, iou_threshold)
print("NMS result:", result)

NMS result: tensor([1, 0])


# Imports

In [10]:
from ultralytics import YOLO
from ultralytics import RTDETR
import os

# Model Training

In [12]:
# load pretrained weights (coco) (n: 20fps, s: 12.5fps)
model = YOLO("yolo11n.pt")

# Only if CPU trianing
# os.environ["CUDA_VISIBLE_DEVICES"] = ""

# fine-tune model
yml_path_full = r"C:\Users\Marlo\Documents\code\QMUL-Societies\QMES\Computer_Vision\datasets\data.yaml"
yml_path_ping_pong = r"C:\Users\Marlo\code\QMUL-Societies\Electronics\Unibots\Computer_Vision\dataset-creation\full-data\data.yaml"
results = model.train(data=yml_path_full, epochs=25, imgsz=640, device='cpu', pretrained=True, fraction=0.2, seed=42)

New https://pypi.org/project/ultralytics/8.3.96 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.36  Python-3.10.11 torch-2.5.1+cu124 CPU (12th Gen Intel Core(TM) i7-12700H)
engine\trainer: task=detect, mode=train, model=yolo11n.pt, data=C:\Users\Marlo\Documents\code\QMUL-Societies\QMES\Computer_Vision\datasets\data.yaml, epochs=25, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=cpu, workers=8, project=None, name=train14, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=42, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=0.2, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, 

train: Scanning C:\Users\Marlo\Documents\code\QMUL-Societies\QMES\Computer_Vision\datasets\labels\train... 220 images, 0 backgrounds, 0 corrupt: 100%|██████████| 220/220 [00:00<00:00, 1392.62it/s]

train: New cache created: C:\Users\Marlo\Documents\code\QMUL-Societies\QMES\Computer_Vision\datasets\labels\train.cache



val: Scanning C:\Users\Marlo\Documents\code\QMUL-Societies\QMES\Computer_Vision\datasets\labels\valid.cache... 100 images, 0 backgrounds, 0 corrupt: 100%|██████████| 100/100 [00:00<?, ?it/s]

Plotting labels to runs\detect\train14\labels.jpg... 


optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001667, momentum=0.9) with parameter groups 81 weight(decay=0.0), 88 weight(decay=0.0005), 87 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 0 dataloader workers
Logging results to runs\detect\train14
Starting training for 25 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


       1/25         0G      1.381      3.312      1.541         33        640:  93%|█████████▎| 13/14 [00:46<00:03,  3.55s/it]


RuntimeError: CUDA error: out of memory
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [ ]:
model.export(format="onnx")

In [ ]:
# try half=True, might not work on some architectures
model.export(format="onnx", imgsz=640, half=False, dynamic=False, simplify=True, batch=1)

In [ ]:
model.export(format='torchscript')

### Export to .ONNX

In [4]:
import torch
import os
import sys
from ultralytics import YOLO

MODEL_PATH = r"C:\Users\Marlo\Documents\code\QMUL-Societies\QMES\unibots\weights\real-world-detector.pt"

IMAGE_WIDTH = 640
IMAGE_HEIGHT = 480

def load_model(model_pth=MODEL_PATH):
    ''' Initialize YOLOv11 Model on CPU '''
    model = None
    try:
        print("Loading YOLOv11 model on CPU...")
        model = YOLO(model_pth)
        model.to("cpu")
        print("YOLOv11 model loaded successfully on CPU.")
    except Exception as e:
        print(f"Error loading YOLOv11 model: {e}")
        sys.exit(1)
    model.eval()
    return model

# Assume IMAGE_HEIGHT and IMAGE_WIDTH are defined (e.g., 480, 640)
dummy_input = torch.randn(1, 3, IMAGE_HEIGHT, IMAGE_WIDTH)

model = load_model(MODEL_PATH)

torch.onnx.export(
    model, 
    dummy_input, 
    "real-world-detector.onnx", 
    opset_version=11,
    input_names=["input"],
    output_names=["output"],
    export_params=True
)

Loading YOLOv11 model on CPU...
YOLOv11 model loaded successfully on CPU.
New https://pypi.org/project/ultralytics/8.3.96 available  Update with 'pip install -U ultralytics'
engine\trainer: task=detect, mode=train, model=C:\Users\Marlo\Documents\code\QMUL-Societies\QMES\unibots\weights\real-world-detector.pt, data=C:\Users\Marlo\code\QMUL-Societies\Electronics\Unibots\Computer_Vision\dataset-creation\full-data\data.yaml, epochs=100, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=cpu, workers=8, project=None, name=train4, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_st

RuntimeError: Dataset 'C://Users/Marlo/code/QMUL-Societies/Electronics/Unibots/Computer_Vision/dataset-creation/full-data/data.yaml' error  'C:\Users\Marlo\code\QMUL-Societies\Electronics\Unibots\Computer_Vision\dataset-creation\full-data\data.yaml' does not exist

# Setup Camera
### Show live camera input

In [3]:
import cv2
import time

source_id = 0

In [ ]:
import cv2
import matplotlib.pyplot as plt

# Open the default camera (usually source 0)
cap = cv2.VideoCapture(0)

if not cap.isOpened():
    print("Error: Could not open camera.")
    exit()

while True:
    ret, frame = cap.read()
    if not ret:
        print("Error: Could not read frame.")
        break

    # Convert BGR (OpenCV format) to RGB (matplotlib format)
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    plt.imshow(frame_rgb)
    plt.title("Camera Feed")
    plt.axis("off")
    plt.show(block=False)
    plt.pause(0.001)
    plt.clf()

    # Optionally, break on a key press if you're running in an interactive environment
    # (Matplotlib doesn't capture key presses like cv2.waitKey)
cap.release()

### Take sample images

In [9]:
import cv2
import os

output_dir = 'captured_frames'
os.makedirs(output_dir, exist_ok=True)

cap = cv2.VideoCapture(0)

if not cap.isOpened():
    print("Cannot open camera")
    exit()

frame_counter = 0
print("Capturing frames. Press Ctrl+C in the console to stop capturing.")

try:
    while True:
        ret, frame = cap.read()
        if not ret:
            print("Can't receive frame (stream end?). Exiting ...")
            break

        # Construct filename and save frame to disk
        filename = os.path.join(output_dir, f'frame_{frame_counter:05d}.jpg')
        cv2.imwrite(filename, frame)
        print(f"Captured image saved as {filename}")
        frame_counter += 1
except KeyboardInterrupt:
    print("Capture stopped by user.")
finally:
    cap.release()
    print("Capture ended. All frames saved in:", output_dir)

Captured image saved as live_cam_images/capture_000.jpg
Captured image saved as live_cam_images/capture_001.jpg
Captured image saved as live_cam_images/capture_002.jpg
Captured image saved as live_cam_images/capture_003.jpg
Captured image saved as live_cam_images/capture_004.jpg
Captured image saved as live_cam_images/capture_005.jpg
Captured image saved as live_cam_images/capture_006.jpg
Captured image saved as live_cam_images/capture_007.jpg
Capture interrupted by user.


# Model Inference

### YOLOv11

In [1]:
from ultralytics import YOLO
import os

In [13]:
model = YOLO(r"C:\Users\Marlo\Documents\code\QMUL-Societies\QMES\Unibots-karl\Computer_Vision\runs\detect\train14\weights\best.pt")  # pretrained YOLO11n model

paths = [r"C:\Users\Marlo\Documents\code\QMUL-Societies\QMES\Unibots-karl\Computer_Vision\dataset-creation\test-data\pic1.jpg",
        r"C:\Users\Marlo\Documents\code\QMUL-Societies\QMES\Unibots-karl\Computer_Vision\dataset-creation\test-data\pic2.jpg",
        r"C:\Users\Marlo\Documents\code\QMUL-Societies\QMES\Unibots-karl\Computer_Vision\dataset-creation\test-data\pic3.jpg",
        r"C:\Users\Marlo\Documents\code\QMUL-Societies\QMES\Unibots-karl\Computer_Vision\dataset-creation\test-data\pic4.jpg"]
paths = [r"C:\Users\Marlo\Documents\code\QMUL-Societies\QMES\Unibots-karl\Computer_Vision\live_cam_images\capture_004.jpg",
         r"C:\Users\Marlo\Documents\code\QMUL-Societies\QMES\Unibots-karl\Computer_Vision\live_cam_images\capture_005.jpg",
         r"C:\Users\Marlo\Documents\code\QMUL-Societies\QMES\Unibots-karl\Computer_Vision\live_cam_images\capture_006.jpg",
         r"C:\Users\Marlo\Documents\code\QMUL-Societies\QMES\Unibots-karl\Computer_Vision\live_cam_images\capture_007.jpg"]

# Run batched inference on a list of images
results = model(paths, conf=0.45, save=True)  # return a list of Results objects

# Process results list
for result in results:
    boxes = result.boxes  # Boxes object for bounding box outputs
    masks = result.masks  # Masks object for segmentation masks outputs
    keypoints = result.keypoints  # Keypoints object for pose outputs
    probs = result.probs  # Probs object for classification outputs
    obb = result.obb  # Oriented boxes object for OBB outputs
    # result.show()  # display to screen
    result.save()  # save to disk
    full_path = os.path.abspath(result.save_dir)
    print(f"Predictions are stored in: {full_path}")


0: 480x640 1 ping-pong-ball, 50.9ms
1: 480x640 2 ping-pong-balls, 50.9ms
2: 480x640 1 ping-pong-ball, 50.9ms
3: 480x640 3 ping-pong-balls, 50.9ms
Speed: 4.4ms preprocess, 50.9ms inference, 1.5ms postprocess per image at shape (1, 3, 480, 640)
Results saved to runs\detect\predict4
Predictions are stored in: c:\Users\Marlo\Documents\code\QMUL-Societies\QMES\Unibots-karl\Computer_Vision\runs\detect\predict4
Predictions are stored in: c:\Users\Marlo\Documents\code\QMUL-Societies\QMES\Unibots-karl\Computer_Vision\runs\detect\predict4
Predictions are stored in: c:\Users\Marlo\Documents\code\QMUL-Societies\QMES\Unibots-karl\Computer_Vision\runs\detect\predict4
Predictions are stored in: c:\Users\Marlo\Documents\code\QMUL-Societies\QMES\Unibots-karl\Computer_Vision\runs\detect\predict4


### Live Camera Input (YOLOv11)

In [ ]:
model = YOLO(r"C:\Users\Marlo\Documents\code\QMUL-Societies\QMES\Unibots-karl\Computer_Vision\runs\detect\train14\weights\best.pt")

results = model(0, stream=True)

for result in results:
    print("start")
    boxes = result.boxes  # Boxes object for bounding box outputs
    masks = result.masks  # Masks object for segmentation masks outputs
    keypoints = result.keypoints  # Keypoints object for pose outputs
    probs = result.probs  # Probs object for classification outputs
    obb = result.obb  # Oriented boxes object for OBB outputs
    # result.show()  # display to screen
    result.save()  # save to disk
    print("finished - wait")
    time.sleep(3)
    print("finished wait - next")

### Unibots Integration

In [ ]:
def ball_detection(img, camera, model):
    x_center_int = None
    try:
        # Get image dimensions
        width = camera.getWidth()
        height = camera.getHeight()

        # Convert the raw image data to a NumPy array
        img_array = np.frombuffer(img, dtype=np.uint8).reshape((height, width, 4))

        # Convert RGBA to RGB by removing the alpha channel
        img_rgb = img_array[:, :, :3]

        # Create a PIL Image from the NumPy array
        image = Image.fromarray(img_rgb)
        image_np = np.array(image)
        print("Model inference starting...")

        # Run the YOLOv11 model on the image
        results = model(image_np, conf=CONFIDENCE_THRESHOLD, save=True)

        # Process and print detected objects
        result = results[0]  # Since there's only one image
        boxes = result.boxes  # Boxes object for bounding box outputs
        if boxes:
            for box in boxes:
                # Extract the bounding box coordinates (x1, y1, x2, y2)
                x1, y1, x2, y2 = box.xyxy[0]
                print(box.xyxy)
                # Calculate the center x position
                x_center = (x1 + x2) / 2

                # Convert to integer
                x_center_int = int(x_center.item())
        else:
            print("Image 0: []")

    except Exception as e:
        print(f"Error during image processing or detection: {e}")
    return x_center_int

### RT-DETR

In [ ]:
# load pretrained weights (coco)
model = RTDETR("rtdetr-l.pt")
model.info()

# fine-tune model
yml_path = r"C:\Users\Marlo\code\QMUL-Societies\Electronics\Unibots\Computer_Vision\dataset-creation\full-data\data.yaml"
results = model.train(data=yml_path, epochs=200, imgsz=640)

# inference
results = model("training-data\comp-vid-24.mp4", save=True)